# Оценка линейного искажающего оператора в задаче восстановления изображений

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import convolve2d as conv2

from skimage import color, data, restoration, io, img_as_float
from skimage.restoration import uft
from scipy.signal import fftconvolve, convolve, convolve2d
from scipy.stats.stats import pearsonr
from numpy.fft import fftn, ifftn

In [ ]:
#interactive graphics
#%matplotlib notebook

In [ ]:
# Prepare data
sz = 11
astro = color.rgb2gray(data.astronaut())

psf = np.ones((sz, sz)) / (sz*sz)
astro_noisy = conv2(astro, psf, 'same')
# Add Noise to Image
#astro_noisy += (np.random.poisson(lam=25, size=astro.shape) - 10) / 255.

In [ ]:
def show_results(original, noisy, result, titles=['Original Data', 'Blurred data', 'Restoration using\nRichardson-Lucy']):
    # Show results
    fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(12, 10))
    plt.gray()

    for a in (ax[0], ax[1], ax[2]):
           a.axis('off')

    ax[0].imshow(original)
    ax[0].set_title(titles[0])

    ax[1].imshow(noisy)
    ax[1].set_title(titles[1])

    ax[2].imshow(result, vmin=noisy.min(), vmax=noisy.max())
    ax[2].set_title(titles[2])


    fig.subplots_adjust(wspace=0.02, hspace=0.2,
                        top=0.9, bottom=0.05, left=0, right=1)
    plt.show()

In [ ]:
def row_correlation(image):
    return pearsonr(image[1:,:].ravel(), image[:-1,:].ravel())[0]
    
def col_correlation(image):
    return pearsonr(image[:,1:].ravel(), image[:,:-1].ravel())[0]

In [ ]:
def desubsample(image, times=1, axises=None):
    if axises is None:
        axises = [i for i, sz in enumerate(image.shape) if sz > 4]
    for axis in axises:
        image = image.repeat(times, axis)#, inplace=True)
    return image

def corelucy(Y, psf, dampar22, wI, readout, subsample, numNSdim, vec, num, eps=1e-9, useFFT=False):
    """
    CORELUCY Accelerated Damped Lucy-Richarson Operator.
    Calculates function that when used with the scaled projected array
    produces the next iteration array that maximizes the likelihood that
    the entire suite satisfies the Poisson statistics.
    """
    if useFFT:
        reBlurred = np.real(ifftn(fftn(Y) * psf)) #Here psf=H = fftn(psf)
    else:
        reBlurred = np.real(convolve2d(Y, psf, 'same'))

    # 1. Resampling if needed
    if subsample != 1: # Bin reBlurred back to the sizeI for non-singleton dims
        #1.Reshape so that the-to-binned dimension separates into two
        #dimensions, with one of them consisting of elements of a single bin.
        reBlurred = np.reshape(reBlurred, vec);

        #2. Bin (==calculate mean) along the first of the-to-binned dimension,
        #that dimension consists of the bin elements. Reshape to get rid off
        for k in num: # new appeared singleton.
            vec.pop(k) #~ vec(k) = [];
            reBlurred = reshape(mean(reBlurred,k),vec);

    # 2. An Estimate for the next step
    reBlurred += readout;
    #reBlurred[reBlurred == 0] = eps;
    AnEstim = wI / (reBlurred + eps);

    # 3. Damping if needed
    if dampar22 == 0: # No Damping
        ImRatio = desubsample(AnEstim, subsample, numNSdim);
    else: # Damping of the image relative to dampar22 = (N*sigma)^2
        gm = 10;
        g = (wI * np.log(AnEstim)+ reBlurred - wI) / dampar22;
        g = np.minimum(g,1);
        G = (g**(gm-1))*(gm-(gm-1)*g);
        ImRatio = 1 + desubsample(G,subsample, numNSdim) * (desubsample(AnEstim, subsample, numNSdim) - 1);
    if useFFT:
        return fftn(ImRatio);
    else:
        return ImRatio

def richardson_lucy_matlab(image, psf, iterations=50, dampar=0, weight=None, readout=0, 
                           subsample=1, eps=1e-16, clip=True, useFFT=False):
    """ Richardson-Lucy deconvolution.

    Parameters
    ----------
    image : ndarray
       Input degraded image (can be N dimensional).
    psf : ndarray
       The point spread function.
    iterations : int
       Number of iterations. This parameter plays the role of
       regularisation.
    """
    #if 'eps' not in globals():
    #    eps = 1e-9
    # to continue restoration
    sizeI = image.shape
    sizePSF = psf.shape
    numNSdim = [i for i, sz in enumerate(sizePSF) if sz > 4] # do not desubsample rgb axis

    assert(not np.all(psf == 0))

    if isinstance(image, list) and len(image) == 4:
        image, prev_image, prev_prev_image, internal = image
    else:
        prev_image = np.ones(image.shape)*.5#image
        prev_prev_image = 0
        internal = np.zeros((image.size*subsample**len(numNSdim),2))
    internal[:,1] = 0

    if weight is None:
        weight = np.ones(image.shape)

    # 1. Prepare OTF
    #~ H = psf2otf(psf, sizeI);
    if psf.shape != image.shape:
        H = uft.ir2tf(psf, image.shape, is_real=False)
    else:
        H = psf

    wI = np.maximum(weight * (readout + image), 0)
    prev_image = desubsample(prev_image, subsample, numNSdim)
    if useFFT:
        scale = np.real(ifftn(np.conj(H)*fftn(desubsample(weight)))) + np.sqrt(eps)
    else:
        scale = np.real(convolve2d(desubsample(weight), psf, 'same')) + np.sqrt(eps)
    del weight

    dampar22 = np.square(dampar)/2

    '''
    prepare vector of dimensions to facilitate the reshaping
    % when the matrix is binned within the iterations.
    '''
    vec = []
    num = []
    if subsample != 1:
        for dim, sz in enumerate(sizeI):
            if sz != 1:
                vec.extend((subsample, sz))
                num.append(dim)
            else:
                vec.append(sz)
    num.reverse()

    # 3 L_R Iterations
    lambd = 2 * np.any(internal != 0)
    correlation_X = [col_correlation(image)]
    correlation_Y = [row_correlation(image)]
    for k in range(lambd, lambd + iterations):

        # 3.a Make an image predictions for the next iteration
        if k > 2:
            lambd = (internal[:,0].T.dot(internal[:,1])) / (internal[:,1].T.dot(internal[:,1]) +eps) # (scalar division)
            lambd = max(min(lambd, 1), 0) # stability enforcement saturation
        Y = np.maximum(prev_image + lambd*(prev_image - prev_prev_image), 0) # plus positivity constraint

        # 3.b Make core for the LR estimation
        if useFFT:
            cc = corelucy(Y, H, dampar22, wI, readout, subsample, numNSdim, vec, num, eps, useFFT)
        else:
            cc = corelucy(Y, psf, dampar22, wI, readout, subsample, numNSdim, vec, num, eps, useFFT)

        # 3.c Determine next iteration image and apply poitivity constraint
        prev_prev_image = prev_image
        if useFFT:
            prev_image = np.maximum(Y * np.real(ifftn(cc * H)) / scale, 0)
        else:
            prev_image = np.maximum(Y * np.real(convolve2d(cc, psf, 'same')) / scale, 0)
        if clip:
            prev_image[prev_image > 1] = 1
            prev_image[prev_image < -1] = eps
        correlation_X.append(col_correlation(prev_image))
        correlation_Y.append(row_correlation(prev_image))
        del cc
        internal[:,1] = internal[:,0]
        internal[:,0] = (prev_image-Y).ravel()
    del wI, H, scale, Y
    return {'image': prev_image, 
            'correlationX': np.array(correlation_X)/(image.size-1), 
            'correlationY': np.array(correlation_Y)/(image.size-1)}

In [ ]:
from itertools import chain
def plot_corr(n, ydata, legend=['row correlation', 'column correlation'], title='Normalized correlation'):
    plt.plot(*list(chain(*[(range(n), y) for y in ydata])))
    plt.grid()
    plt.title(title)
    plt.xlabel('n iterations')
    plt.legend(legend)
    plt.show()

In [ ]:
# Restore Image using Richardson-Lucy algorithm:
iterations = 20
deconv = richardson_lucy_matlab(astro_noisy, psf, iterations=iterations, eps=1e-5)
show_results(astro, astro_noisy, deconv['image'])
plot_corr(iterations+1, [deconv['correlationX'], 
                         deconv['correlationY']])

In [ ]:
liftingbody = img_as_float(io.imread('liftingbody.png'))
lifting_blurred = conv2(liftingbody, psf, 'same')

In [ ]:
# Restore Image using Richardson-Lucy algorithm my:
iterations = 20
deconv = richardson_lucy_matlab(lifting_blurred, psf, iterations=iterations, eps=1e-5, useFFT=True)
show_results(liftingbody, lifting_blurred, deconv['image'])
plot_corr(iterations+1, [deconv['correlationX'], 
                         deconv['correlationY']])

// matlab original for gen psf (curved line shift)
```python
def gen_psf():
    p0 = np.random.randint(6, 25)
    p1 = np.random.randint(6, 21)

    x = np.random.randint(0, p0, (1,4))#sort(randi([0 p0], 1, 4));
    y = np.random.randint(0, p1, (1,4))

    pt1 = [x[0]; y[0]]
    pt2 = [x[1]; y[1]]
    pt3 = [x[2]; y[2]]
    pt4 = [x[3]; y[3]]

    t = list(range(0, p0+1))
    pts = kron((1-t).^3,pt1) + kron(3*(1-t).^2.*t,pt2) + kron(3*(1-t).*t.^2,pt3) + kron(t.^3,pt4);
    x1 = 1:p0;
    y1 = round(pts(2, :));
    y1 = y1 - min(y1) + 1;
    %plot(x1,y1);
    pp = max(y1);
    PSF = zeros(p0, pp);

    for ind_x = 1:length(x1)
        yy = max(length(x1) - y1(ind_x), 1);
        PSF(yy, ind_x) = rand();
    end

    PSF = PSF./(sum(PSF(:)) + eps*p0*p0);
```

## Криволинейный оператор смаза

In [ ]:
def curved_psf():
    eps = 1e-16
    p0 = np.random.randint(6, 25)
    p1 = np.random.randint(6, 21)

    x = np.random.randint(0, p0, 4)#sort(randi([0 p0], 1, 4));
    y = np.random.randint(0, p1, 4)

    pt1 = [[x[0]], [y[0]]]
    pt2 = [[x[1]], [y[1]]]
    pt3 = [[x[2]], [y[2]]]
    pt4 = [[x[3]], [y[3]]]

    t = np.linspace(0, 1, p0)
    pts = np.kron((1-t) ** 3, pt1) + np.kron(3*(1-t)**2 * t, pt2) + np.kron(3 * (1-t) * t**2, pt3) + np.kron(t**3, pt4)
    x1 = np.arange(1, p0+1)
    y1 = np.round(pts[1, :]).astype(np.int64)
    y1 = y1 - min(y1) + 1
    #plot(x1,y1)
    pp = np.max(y1)
    PSF = np.zeros((p0, len(x1)))

    print(len(x1), PSF.shape)

    for ind_x in range(len(x1)):
        yy = max(len(x1) - y1[ind_x], 1)
        PSF[yy, ind_x] = np.random.rand()

    PSF = PSF/(sum(PSF.ravel()) + eps*p0*p0)
    return PSF

In [ ]:
psf = curved_psf()
lifting_blurred = conv2(liftingbody, psf, 'same')

In [ ]:
# Restore Image using Richardson-Lucy algorithm my:
iterations = 3
deconv = richardson_lucy_matlab(lifting_blurred, psf, iterations=iterations, eps=1e-5)
show_results(liftingbody, lifting_blurred, deconv['image'])
plot_corr(iterations+1, [deconv['correlationX'], 
                         deconv['correlationY']])

## Линейный оператор смаза

In [ ]:
from skimage.draw import line_aa
from math import sin, cos, pi
def motion_blur_psf(length, angle):
    rr, cc, val = line_aa(0, 0, int(length*cos(angle)), int(length*sin(angle)))
    psf = np.zeros((max(rr)+1, max(cc)+1))
    psf[rr, cc] = val
    #psf[0, 0] = psf[0, 0]/2
    #psf[-1, -1] = psf[-1, -1]/2
    return psf/np.sum(psf)

In [ ]:
shift = 11
psf = motion_blur_psf(shift, pi/3)
lifting_blurred = conv2(liftingbody, psf, 'same')
iterations = 20
deconv = richardson_lucy_matlab(lifting_blurred, psf, iterations=iterations, eps=1e-5, clip=False)
show_results(liftingbody, lifting_blurred, deconv['image'])
plot_corr(iterations+1, [deconv['correlationX'], 
                         deconv['correlationY']])
correlation_X = pearsonr(liftingbody.ravel('C')[:-1], liftingbody.ravel('C')[1:])
correlation_Y = pearsonr(liftingbody.ravel('F')[:-1], liftingbody.ravel('F')[1:])
print(correlation_X, correlation_Y)

## Неверная psf

In [ ]:
psf_wrong = motion_blur_psf(shift, pi/2)
iterations = 20
deconv_wrong = richardson_lucy_matlab(lifting_blurred, psf_wrong, iterations=iterations, eps=1e-5)
show_results(liftingbody, lifting_blurred, deconv_wrong['image'])
plot_corr(iterations+1, [deconv['correlationX'], 
                         deconv['correlationY'],
                         deconv_wrong['correlationX'],
                         deconv_wrong['correlationY']], 
          legend=['row correlation', 'column correlation', 'row correlateion(wrong psf)', 'column correlation(wrong psf)'])

## Использование параметра dampar

In [ ]:
def find_noise(image):
    N, M = image.shape
    F = np.fft.fftn(image)
    part = .02
    area = F[int(N/2-N*part):int(N/2+N*part), int(M/2-M*part):int(M/2+M*part)]
    np.std
    np.var
    msv = np.sqrt(np.mean(np.abs(area) ** 2) / (N*M))
    I_res = np.fft.ifftn(F-msv)
    NSR = msv**2 / np.var(I_res)
    return (msv, NSR)
s_n, S_find = find_noise(liftingbody)

In [ ]:
s_n

In [ ]:
psf = motion_blur_psf(shift, pi/3)
lifting_blurred = conv2(liftingbody, psf, 'same')
iterations = 40
deconv = richardson_lucy_matlab(lifting_blurred, psf, iterations=iterations, eps=1e-5, clip=False, dampar=s_n)
show_results(liftingbody, lifting_blurred, deconv['image'])
plot_corr(iterations+1, [deconv['correlationX'], 
                         deconv['correlationY']])

### dampar и неверная psf

In [ ]:
psf_wrong = motion_blur_psf(shift, pi/2)
#iterations = 40
deconv_wrong = richardson_lucy_matlab(lifting_blurred, psf_wrong, iterations=iterations, eps=1e-5, dampar=s_n)
show_results(liftingbody, lifting_blurred, deconv_wrong['image'])
plot_corr(iterations+1, [deconv['correlationX'], 
                         deconv['correlationY'],
                         deconv_wrong['correlationX'],
                         deconv_wrong['correlationY']], 
          legend=['row correlation', 'column correlation', 'row correlateion(wrong psf)', 'column correlation(wrong psf)'])

```
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter


w, h = lifting_blurred.shape
x = np.arange(w)
y = np.arange(h)
X, Y = np.meshgrid(x, y)

fig = plt.figure(figsize=(10,10))
ax = fig.gca(projection='3d')
# Plot the surface.
surf = ax.plot_surface(X, Y, F, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)

# Customize the z axis.
#ax.set_zlim(0, 1.01)
#ax.zaxis.set_major_locator(LinearLocator(10))
#ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

# Add a color bar which maps values to colors.
#fig.colorbar(surf, shrink=0.5, aspect=5)

plt.show()
```

## Кепстр
$$K = F^{-1}\{log(1+\left|F\{I\}\right|)\}$$

In [ ]:
from skimage.filters import gaussian

b_clip = 50
K = np.fft.ifftn(100*np.log(1+np.abs(np.fft.fftn(lifting_blurred))))[b_clip:-b_clip,b_clip:-b_clip]
K = gaussian(np.abs(K))
#N,M = K.shape
#mask = np.ones((N, M))
#mask[1:3, 1:3] = 0
#K *= mask

In [ ]:
#%matplotlib notebook
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter


w, h = K.shape
x = np.arange(w)
y = np.arange(h)
X, Y = np.meshgrid(x, y)

fig = plt.figure(figsize=(10,10))
ax = fig.gca(projection='3d')
# Plot the surface.
H = 1
surf = ax.plot_surface(X, Y, np.clip(np.abs(K), 0, H), cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)

# Customize the z axis.
#ax.set_zlim(0, H + H/20)
#ax.zaxis.set_major_locator(LinearLocator(10))
#ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

# Add a color bar which maps values to colors.
#fig.colorbar(surf, shrink=0.5, aspect=5)

plt.show()

## Уточнение искажающего оператора
$$\varepsilon = ||\tilde{I}\oplus\tilde{h}-I_0|| \to \min_{(x,y)}$$
Пусть $$\tilde{I}\oplus(\tilde{h}+\tilde{dh})=I_0$$
Тогда $$\tilde{I}\oplus\tilde{dh}=I_0-\tilde{I}\oplus\tilde{h}$$
Получим задачу аналогичную исходной($\tilde{I}\oplus h +\eta = I_0$)

In [ ]:
img_diff = liftingbody - convolve2d(deconv['image'], psf, mode='same') # Утечка
deconv_psf = richardson_lucy_matlab(img_diff, deconv['image'], iterations=iterations, eps=1e-5, dampar=s_n, useFFT=True)
psf_new = deconv_psf['image']
deconv_upd = richardson_lucy_matlab(lifting_blurred, psf_new, iterations=iterations, eps=1e-5, dampar=s_n, useFFT=True)
show_results(liftingbody, lifting_blurred, deconv_upd['image'],
             titles=['blurred', 'restored', 'restored with\nnew psf'])
plot_corr(iterations+1, [deconv['correlationX'], 
                         deconv['correlationY'],
                         deconv_upd['correlationX'],
                         deconv_upd['correlationY']], 
          legend=['row correlation', 'column correlation', 'row correlateion(new psf)', 'column correlation(new psf)'])